### Notebook to demonstrate Image Classification workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### Sample prediction for an Image Classification model
<img align="center" src="../example_images/sample_image_classification.jpg">

### The workflow in a nutshell

- Creating a dataset
- Upload dataset to the service
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Prune, retrain
    - Export
    - TAO-Deploy
    - Inference on TAO

### Table of contents

1. [Create datasets ](#head-1)
1. [List the created datasets](#head-2)
1. [Create model ](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [View hyperparameters that are enabled by default](#head-8)
1. [Set AutoML related configurations](#head-9)
1. [Actions](#head-10)
1. [Train](#head-11)
1. [Evaluate](#head-12)
1. [Optimize: Apply specs for prune](#head-14)
1. [Optimize: Apply specs for retrain](#head-15)
1. [Optimize: Run actions](#head-16)
1. [Export](#head-17)
1. [TRT Engine generation using TAO-Deploy](#head-19)
1. [TAO inference](#head-20)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a model_name in FIXME 1
2. Assign a workdir in FIXME 2
3. Assign the ip_address and port_number in FIXME 3 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
4. Assign the ngc_api_key variable in FIXME 4
5. (Optional) Enable AutoML if needed in FIXME 5
6. Choose between default and custom dataset in FIXME 6
7. Assign path of DATA_DIR in FIXME 7
8. Choose between Bayesian and Hyperband automl_algorithm in FIXME 8 (If automl was enabled in FIXME5)

In [ ]:
# Define model_name workspaces and other variables
# Available models (#FIXME 1):
# 1. classification_pyt - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 2. classification_tf1 - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 3. classification_tf2 - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification_tf2.html
# 4. multitask_classification - https://docs.nvidia.com/tao/tao-toolkit/text/multitask_image_classification.html
# classification is the same as multi-class classification

model_name = "multitask_classification" # FIXME1 (Add the model name from the above mentioned list)

In [ ]:
workdir = "workdir_classification" # FIXME2
host_url = "http://<ip_address>:<port_number>" # FIXME3 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME4 example: (Add NGC API key)

In [ ]:
automl_enabled = False # FIXME5 set to TRUE if you want to run automl for the model chosen in the previous cell

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

**For multi-class classification:**

We will be using the pascal `VOC dataset` for the tutorial. To find more details please visit [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit). Please download the [dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar) to the environment variable `$DATA_DIR`.

**If using custom dataset; it should follow this dataset structure, and skip running** "**Split dataset into train and val sets**"
```
DATA_DIR
├── classes.txt
├── images_test
│   ├── class_name_1
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
│   │   ├── ...
|   |   ... 
│   └── class_name_n
│       ├── image_name_3.jpg
│       ├── image_name_4.jpg
│       ├── ...
├── images_train
│   ├── class_name_1
│   │   ├── image_name_5.jpg
│   │   ├── image_name_6.jpg
|   |   ...
│   └── class_name_n
│       ├── image_name_7.jpg
│       ├── image_name_8.jpg
│       ├── ...
|
└── images_val
    ├── class_name_1
    │   ├── image_name_9.jpg
    │   ├── image_name_10.jpg
    │   ├── ...
    |   ...
    └── class_name_n
        ├── image_name_11.jpg
        ├── image_name_12.jpg
        ├── ...
```
- Each class name folder should contain the images corresponding to that class
- Same class name folders should be present across images_test, images_train and images_val
- classes.txt is a file which contains the names of all classes (each name in a separate line)

**For multi-task classification:**

We will be using the `Fashion Product Images (Small)` for the tutorial. This dataset is available on Kaggle.In this tutorial, our trained classification network will perform three tasks: article category classification, base color classification and target season classification.

To download the dataset, you will need a Kaggle account. After login, you can download the dataset zip file [here](https://www.kaggle.com/paramaggarwal/fashion-product-images-small). The downloaded file is archive.zip with a subfolder called myntradataset. Unzip contents in this subfolder to your workdir created in the cell above and you should have a folder called images and a CSV file called styles.csv

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   |   ├── ...
├── styles.csv
```

In [ ]:
dataset_to_be_used = "default" #FIXME6 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = model_name # FIXME7
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
if dataset_to_be_used == "default":
    if "classification_" in model_name:
        if not os.path.exists(os.path.join(DATA_DIR,"VOCtrainval_11-May-2012.tar")):
            print("Download VOC tar data into ", DATA_DIR)
        else:
            !tar -xf $DATA_DIR/VOCtrainval_11-May-2012.tar -C $DATA_DIR
    elif model_name == "multitask_classification":
        if not os.path.exists(os.path.join(DATA_DIR,"archive.zip")):
            print(f"Download Fashion zip data into ", DATA_DIR)
        else:
            !unzip -uq $DATA_DIR/archive.zip -d $DATA_DIR/

### Split dataset into train and val sets

In [ ]:
# Check the dataset is present
if "classification_" in model_name and dataset_to_be_used == "default":
    !if [ ! -d $DATA_DIR/VOCdevkit ]; then echo 'Images folder NOT found.'; else echo 'Found images folder.';fi
    !rm -rf $DATA_DIR/split
elif model_name == "multitask_classification":
    !if [ ! -d $DATA_DIR/images ]; then echo 'Images folder NOT found.'; else echo 'Found images folder.';fi
    !if [ ! -f $DATA_DIR/styles.csv ]; then echo 'CSV file NOT found.'; else echo 'Found CSV file.';fi
    # Create subdirectories and remove existing files in them
    !mkdir -p $DATA_DIR/images_train && rm -rf $DATA_DIR/images_train/*
    !mkdir -p $DATA_DIR/images_val && rm -rf $DATA_DIR/images_val/*
    !mkdir -p $DATA_DIR/images_test && rm -rf $DATA_DIR/images_test/*

In [ ]:
# Split dataset into train and val sets
!python3 -m pip install numpy
!python3 -m pip install pandas==1.5.1
if "classification_" in model_name and dataset_to_be_used == "default":
    !python3 -m pip install tqdm
    from os.path import join as join_path
    import os
    import glob
    import re
    import shutil

    DATA_DIR=os.environ.get('DATA_DIR')
    source_dir = join_path(DATA_DIR, "VOCdevkit/VOC2012")
    target_dir = join_path(DATA_DIR, "formatted")

    suffix = '_trainval.txt'
    classes_dir = join_path(source_dir, "ImageSets", "Main")
    images_dir = join_path(source_dir, "JPEGImages")
    classes_files = glob.glob(classes_dir+"/*"+suffix)
    class_names = []
    for file in classes_files:
        # get the filename and make output class folder
        classname = os.path.basename(file)
        if classname.endswith(suffix):
            classname = classname[:-len(suffix)]
            target_dir_path = join_path(target_dir, classname)
            if not os.path.exists(target_dir_path):
                os.makedirs(target_dir_path)
        else:
            continue
        print(classname)
        class_names.append(classname)

        with open(file) as f:
            content = f.readlines()

        for line in content:
            tokens = re.split('\s+', line)
            if tokens[1] == '1':
                # copy this image into target dir_path
                target_file_path = join_path(target_dir_path, tokens[0] + '.jpg')
                src_file_path = join_path(images_dir, tokens[0] + '.jpg')
                shutil.copyfile(src_file_path, target_file_path)

    from random import shuffle
    from tqdm import tqdm

    DATA_DIR=os.environ.get('DATA_DIR')
    SOURCE_DIR=os.path.join(DATA_DIR, 'formatted')
    TARGET_DIR=os.path.join(DATA_DIR,'split')
    # list dir
    print(os.walk(SOURCE_DIR))
    dir_list = next(os.walk(SOURCE_DIR))[1]
    # for each dir, create a new dir in split
    for dir_i in tqdm(dir_list):
        newdir_train = os.path.join(TARGET_DIR, 'images_train', dir_i)
        newdir_val = os.path.join(TARGET_DIR, 'images_val', dir_i)
        newdir_test = os.path.join(TARGET_DIR, 'images_test', dir_i)

        if not os.path.exists(newdir_train):
                os.makedirs(newdir_train)
        if not os.path.exists(newdir_val):
                os.makedirs(newdir_val)
        if not os.path.exists(newdir_test):
                os.makedirs(newdir_test)

        img_list = glob.glob(os.path.join(SOURCE_DIR, dir_i, '*.jpg'))
        # shuffle data
        shuffle(img_list)

        for j in range(int(len(img_list)*0.7)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_train', dir_i))

        for j in range(int(len(img_list)*0.7), int(len(img_list)*0.8)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_val', dir_i))

        for j in range(int(len(img_list)*0.8), len(img_list)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_test', dir_i))

    with open(f'{DATA_DIR}/classes.txt', 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")

    shutil.copy2(f'{DATA_DIR}/classes.txt',TARGET_DIR)
    print('Done splitting dataset.')

elif model_name == "multitask_classification" and dataset_to_be_used == "default":
    !python3 -m pip install numpy
    !python3 -m pip install pandas==1.5.1
    import os
    import shutil
    import numpy as np
    import pandas as pd

    df = pd.read_csv(os.environ['DATA_DIR'] + '/styles.csv', on_bad_lines='skip')
    df = df[['id', 'baseColour', 'subCategory', 'season']]
    df = df.dropna()
    category_cls = df.subCategory.value_counts()[:10].index # 10-class multitask-classification
    season_cls = ['Spring', 'Summer', 'Fall', 'Winter'] # 4-class multitask-classification
    color_cls = df.baseColour.value_counts()[:11].index # 11-class multitask-classification

    # Get all valid rows
    df = df[df.subCategory.isin(category_cls) & df.season.isin(season_cls) & df.baseColour.isin(color_cls)]
    df.columns = ['fname', 'base_color', 'category', 'season']
    df.fname = df.fname.astype(str)
    df.fname = df.fname + '.jpg'

    # remove entries whose image file is missing
    all_img_files = os.listdir(os.environ['DATA_DIR'] + '/images')
    df = df[df.fname.isin(all_img_files)]

    idx = np.arange(len(df))
    np.random.shuffle(idx)

    train_split_idx = int(len(df)*0.8)
    train_df = df.iloc[idx[:train_split_idx]]
    val_df = df.iloc[idx[train_split_idx:train_split_idx+(len(df) // 10)]]
    test_df = df.iloc[idx[train_split_idx+(len(df) // 10):]]

    # Add a simple sanity check
    assert len(train_df.season.unique()) == 4 and len(train_df.base_color.unique()) == 11 and \
        len(train_df.category.unique()) == 10, 'Training set misses some classes, re-run this cell!'
    assert len(val_df.season.unique()) == 4 and len(val_df.base_color.unique()) == 11 and \
        len(val_df.category.unique()) == 10, 'Validation set misses some classes, re-run this cell!'
    assert len(test_df.season.unique()) == 4 and len(test_df.base_color.unique()) == 11 and \
        len(test_df.category.unique()) == 10, 'Test set misses some classes, re-run this cell!'

    for image_name in train_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
        shutil.copy(source_file_name, destination_file_name)

    for image_name in val_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
        shutil.copy(source_file_name, destination_file_name)
        destination_file_name = os.path.join(DATA_DIR, "images_val", image_name)
        shutil.copy(source_file_name, destination_file_name)

    for image_name in test_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_test", image_name)
        shutil.copy(source_file_name, destination_file_name)

    # save processed data labels
    train_df.to_csv(os.environ['DATA_DIR'] + '/train.csv', index=False)
    val_df.to_csv(os.environ['DATA_DIR'] + '/val.csv', index=False)

### Verify the dataset split

In [ ]:
# verify
if "classification_" in model_name:
    !if [ ! -d $DATA_DIR/split/images_train ]; then echo 'train folder NOT found.'; else echo 'Found train images folder.';fi
    !if [ ! -d $DATA_DIR/split/images_val ]; then echo 'val folder NOT found.'; else echo 'Found val images folder.';fi
    !if [ ! -d $DATA_DIR/split/images_test ]; then echo 'test folder NOT found.'; else echo 'Found test images folder.';fi
    !if [ ! -f $DATA_DIR/split/classes.txt ]; then echo 'classes.txt not found'; else echo 'Found classes.txt.';fi
elif model_name == "multitask_classification":
    import pandas as pd

    print("Number of images in the train set. {}".format(
        len(pd.read_csv(os.environ['DATA_DIR'] + '/train.csv'))
    ))
    print("Number of images in the validation set. {}".format(
        len(pd.read_csv(os.environ['DATA_DIR'] + '/val.csv'))
    ))

### Create Tar files to upload

In [ ]:
if "classification_" in model_name:
    !tar -C $DATA_DIR/split/ -czf classification_train.tar.gz images_train classes.txt
    !tar -C $DATA_DIR/split/ -czf classification_val.tar.gz images_val classes.txt
    !tar -C $DATA_DIR/split/ -czf classification_test.tar.gz images_test classes.txt
elif model_name == "multitask_classification":
    !tar -C $DATA_DIR/ -czf mt_classification_train.tar.gz images_train train.csv val.csv
    !tar -C $DATA_DIR/ -czf mt_classification_val.tar.gz images_val val.csv
    !tar -C $DATA_DIR/ -czf mt_classification_test.tar.gz images_test

In [ ]:
if "classification_" in model_name:
    train_dataset_path =  "classification_train.tar.gz"
    eval_dataset_path = "classification_val.tar.gz"
    test_dataset_path = "classification_test.tar.gz"
elif model_name == "multitask_classification":
    train_dataset_path =  "mt_classification_train.tar.gz"
    eval_dataset_path = "mt_classification_val.tar.gz"
    test_dataset_path = "mt_classification_test.tar.gz"

### Create and upload datasets

In [ ]:
# Create train dataset
ds_type = "image_classification"
if model_name == "classification_pyt":
    ds_format = model_name
elif "classification_" in model_name:
    ds_format = "default"
elif model_name == "multitask_classification":
    ds_format = "custom"

data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
ds_type = "image_classification"
if model_name == "classification_pyt":
    ds_format = model_name
else:
    ds_format = "default"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
 # Create testing dataset for inference
ds_type = "image_classification"
ds_format = "raw"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
test_dataset_id = response.json()["id"]

In [ ]:
# Upload
files = [("file",open(test_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{test_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
network_arch = model_name
if "classification" in network_arch:
    encode_key = "nvidia_tlt"
else:
    encode_key = "tlt_encode"
data = json.dumps({"network_arch":network_arch,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id,
                       "inference_dataset":test_dataset_id,
                       "calibration_dataset":dataset_id}

data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

Search for the PTM on NGC for the Classification model chosen

In [ ]:
# List all pretrained models for the chosen network architecture
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

for rsp in response_json:
    if rsp["network_arch"] == network_arch:
        if "encryption_key" not in rsp.keys():
            print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

In [ ]:
# Assigning pretrained models to different classification models
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"classification_tf1" : "pretrained_classification:resnet18",
                  "classification_tf2" : "pretrained_classification_tf2:efficientnet_b0",
                  "classification_pyt" : "pretrained_fan_classification_imagenet:fan_hybrid_tiny",
                  "multitask_classification" : "pretrained_classification:resnet10"}
no_ptm_models = set([])

In [ ]:
# Get pretrained model for classification
if model_name not in no_ptm_models:
    model_list = f"{base_url}/model"
    response = requests.get(model_list, headers=headers)

    response_json = response.json()

    # Search for ptm with given ngc path
    ptm = []
    for rsp in response_json:
        if rsp["network_arch"] == model_name and rsp["ngc_path"].endswith(pretrained_map[model_name]):
            ptm_id = rsp["id"]
            ptm = [ptm_id]
            print("Metadata for model with requested NGC Path")
            print(rsp)
            break

In [ ]:
if model_name not in no_ptm_models:
    ptm_information = {"ptm":ptm}
    data = json.dumps(ptm_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-8"></a>

In [ ]:
if automl_enabled:
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/train/schema"
    response = requests.get(endpoint, headers=headers)
    specs = response.json()["automl_default_parameters"]
    print(json.dumps(specs, sort_keys=True, indent=4))

### Set AutoML related configurations <a class="anchor" id="head-9"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters:

[Classification TF1](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_tf1/classification_tf1%20-%20train.csv), 
[Classification TF2](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_tf2/classification_tf2%20-%20train.csv), 
[Classification Pytorch](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_pyt/classification_pyt%20-%20train.csv), 
[Multitask classification](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/multitask_classification/multitask_classification%20-%20train.csv)

In [ ]:
if automl_enabled:
    # Choose automl algorithm between "Bayesian" and "HyperBand".
    automl_algorithm="Bayesian" # FIXME8 example: Bayesian/HyperBand
    if model_name == "classification_pyt":
        metric = "loss"
    else:
        metric = "kpi" 

    additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
    remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

    automl_information = {"automl_enabled":automl_enabled,
                          "automl_algorithm":automl_algorithm,
                          "metric":metric,
                          "epoch_multiplier": 1, # Will be considered for Hyperband only
                          "automl_add_hyperparameters":str(additional_automl_parameters),
                          "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                         }
    data = json.dumps(automl_information)

    endpoint = f"{base_url}/model/{model_id}"

    response = requests.patch(endpoint, data=data, headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Actions <a class="anchor" id="head-10"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### Train <a class="anchor" id="head-11"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
# Example for multitask-classification (for each network the parameter key might be different)
if model_name == "multitask_classification":
    specs["training_config"]["num_epochs"] = 10
    specs["gpus"] = 1
# Example for classification_pyt
elif model_name == "classification_pyt":
    specs["train"]["train_config"]["runner"]["max_epochs"] = 40
    specs["train"]["num_gpus"] = 1
    specs["gpus"] = 1
# Example for classification_tf1
elif model_name == "classification_tf1":
    specs["train_config"]["n_epochs"] = 80
    specs["gpus"] = 1
# Example for classification_tf2
elif model_name == "classification_tf2":
    specs["train"]["num_epochs"] = 80
    specs["gpus"] = 1

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
# For automl: Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    if "error_desc" in response.json().keys() and response.json()["error_desc"] in ("Job not found", "No AutoML run found"):
        print("Job is being created")
        time.sleep(5)
        continue
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     job_id = job_map['train']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status by repeatedly running this cell' cell above (4th cell above from this cell)
# if automl_enabled:
#     job_id = job_map['train']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/resume"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
# Download job contents once the above job shows "Done" status
# Download output of multiclass_classification train (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# View the checkpoints generated for the training job and for automl jobs, in addition view: best performing model's config and the results of all automl experiments

if automl_enabled:
    !python3 -m pip install pandas==1.5.1
    import pandas as pd
    model_downloaded_path = f"{model_downloaded_path}/best_model"

if os.path.exists(model_downloaded_path):        
    #List the binary model file
    print("\nCheckpoints for the training experiment")
    if os.path.exists(model_downloaded_path+"/train/weights") and len(os.listdir(model_downloaded_path+"/train/weights")) > 0:
        print(f"Folder: {model_downloaded_path}/train/weights")
        print("Files:", os.listdir(model_downloaded_path+"/train/weights"))
    elif os.path.exists(model_downloaded_path+"/weights") and len(os.listdir(model_downloaded_path+"/weights")) > 0:
        print(f"Folder: {model_downloaded_path}/weights")
        print("Files:", os.listdir(model_downloaded_path+"/weights"))
    else:
        print(f"Folder: {model_downloaded_path}")
        print("Files:", os.listdir(model_downloaded_path))

    if automl_enabled:
        experiment_artifacts = json.load(open(f"{model_downloaded_path}/controller.json","r"))
        data_frame = pd.DataFrame(experiment_artifacts)
        # Print experiment id/number and the corresponding result
        print("\nResults of all experiments")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
            print(data_frame[["id","result"]])

### Evaluate <a class="anchor" id="head-12"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/evaluate/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Modify specs dictionary to change any config parameters

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/evaluate"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["evaluate"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['evaluate']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:    
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Optimize <a class="anchor" id="head-13"></a>

- We optimize the trained model by pruning and retraining in the following cells

### Apply specs for prune <a class="anchor" id="head-14"></a>

In [ ]:
if model_name != "classification_pyt":
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/prune/schema"

    response = requests.get(endpoint, headers=headers)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
if model_name != "classification_pyt":
    # Apply changes to specs dictionary if required here
    if model_name == "classification_tf2":
        specs["prune"]["byom_model_path"] = ""

In [ ]:
if model_name != "classification_pyt":
    # Post spec
    data = json.dumps(specs)

    endpoint = f"{base_url}/model/{model_id}/specs/prune"

    response = requests.post(endpoint,data=data,headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Apply specs for retrain <a class="anchor" id="head-15"></a>

In [ ]:
if model_name != "classification_pyt":
    # Get default spec schema
    endpoint = f"{base_url}/model/{model_id}/specs/retrain/schema"

    response = requests.get(endpoint, headers=headers)

    print(response)
    #print(response.json()) ## Uncomment for verbose schema
    specs = response.json()["default"]
    print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
if model_name != "classification_pyt":
    # Override any of the parameters listed in the previous cell as required
    # Example for multitask-classification (for each network the parameter key might be different)
    if model_name == "multitask_classification":
        specs["training_config"]["num_epochs"] = 10
        specs["gpus"] = 1
    # Example for classification_tf1
    elif model_name == "classification_tf1":
        specs["train_config"]["n_epochs"] = 80
        specs["gpus"] = 1
    # Example for classification_tf2
    elif model_name == "classification_tf2":
        specs["train"]["num_epochs"] = 80
        specs["gpus"] = 1

In [ ]:
if model_name != "classification_pyt":
    # Post spec
    data = json.dumps(specs)

    endpoint = f"{base_url}/model/{model_id}/specs/retrain"

    response = requests.post(endpoint,data=data,headers=headers)

    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))

### Run Actions <a class="anchor" id="head-16"></a>

We use the API's job chaining feature to prune, retrain and evaluate the retrained model

In [ ]:
if model_name != "classification_pyt":
    # Run actions
    parent = job_map["train"]
    actions = ["prune","retrain","evaluate"]
    data = json.dumps({"job":parent,"actions":actions})

    endpoint = f"{base_url}/model/{model_id}/job"

    response = requests.post(endpoint, data=data, headers=headers)

    print(response)
    print(response.json())

    job_map["prune"] = response.json()[0]
    job_map["retrain"] = response.json()[1]
    job_map["eval_retrain"] = response.json()[2]
    print(job_map)

In [ ]:
if model_name != "classification_pyt":
    # Monitor job status by repeatedly running this cell (prune)
    job_id = job_map['prune']
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:    
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
if model_name != "classification_pyt":
    # Monitor job status by repeatedly running this cell (retrain)
    job_id = job_map['retrain']
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:    
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
if model_name != "classification_pyt":
    # Monitor job status by repeatedly running this cell (evaluate)
    job_id = job_map['eval_retrain']
    endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    while True:    
        clear_output(wait=True)
        response = requests.get(endpoint, headers=headers)
        print(response)
        print(response.json())
        if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
            break
        time.sleep(15)

In [ ]:
# if model_name != "classification_pyt":
#    # Optional cancel job - for jobs that are pending/running (retrain)

#     job_id = job_map['retrain']
#     endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

#     response = requests.post(endpoint, headers=headers)

#     print(response)
#     print(response.json())

In [ ]:
# if model_name != "classification_pyt":
    # # Optional delete job - for jobs that are error/done (retrain)

    # job_id = job_map['retrain']
    # endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

    # response = requests.delete(endpoint, headers=headers)

    # print(response)
    # print(response.json())

### Export <a class="anchor" id="head-17"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to spec dictionary if required

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["export"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['export']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["export"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Look for the generated .onnx file
!ls {model_downloaded_path}

### TRT Engine generation using TAO-Deploy <a class="anchor" id="head-19"></a>

- Here, we use the exported model to generate trt engine

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/gen_trt_engine/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
if model_name == "classification_tf2":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
elif model_name == "classification_pyt":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
else:
    specs["data_type"] = "int8"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/gen_trt_engine"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["export"]
actions = ["gen_trt_engine"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["model_gen_trt_engine"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['model_gen_trt_engine']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TAO inference <a class="anchor" id="head-20"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs you want to modify

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_tlt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_tlt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_tlt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
# Inference output must be here
!ls {inference_out_path}/

In [ ]:
# Print Classification results
if model_name == "classification_tf1":
    !cat {inference_out_path}/result.csv
elif "classification_" in model_name:
    !cat {inference_out_path}/inference/result.csv
elif model_name == "multitask_classification":
    !cat {inference_out_path}/result.txt

### TRT inference <a class="anchor" id="head-21"></a>

- no need to change the specs since we already uploaded it at the tlt inference step

In [ ]:
# Run action
parent = job_map["model_gen_trt_engine"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_trt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_trt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_tlt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
# Inference output must be here
!ls {inference_out_path}/

In [ ]:
# Print Classification results
if model_name == "classification_tf1":
    !cat {inference_out_path}/result.csv
elif "classification_" in model_name:
    !cat {inference_out_path}/inference/result.csv
elif model_name == "multitask_classification":
    !cat {inference_out_path}/result.txt